In [2]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

path = "C:/Users/Michael/Desktop/Berkeley/FinTech/DR/fintech_DR/images"
filenames = os.listdir(path)

with open('C:/Users/Michael/Desktop/Berkeley/FinTech/DR/fintech_DR/labels.json') as label:
    labels = json.load(label)


file_df = pd.DataFrame(filenames, columns = ["file_name"])
label_temp_df = pd.DataFrame(labels, columns = ["file_name", "DR_stage"])
label_df = pd.merge(file_df, label_temp_df, on = ["file_name", "file_name"])


images = []

for i in range(len(filenames)):              
    temp = cv2.imread(path + "/" + filenames[i])
    temp = cv2.resize(temp, (512,512))
    images.append(temp)

    
images_arr = np.array(images)



In [3]:
#### training and testing set split
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical

x_train, x_test, y_train, y_test = train_test_split(images_arr, label_df.iloc[:,1], test_size=0.2)

# Onehot encoding for catogory
y_trainOneHot = to_categorical(y_train)
y_testOneHot = to_categorical(y_test)

In [4]:
### CNN 
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D




model = Sequential()

model.add(Conv2D(filters = 10, kernel_size = (3,3), input_shape = (512,512,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(64, activation = "relu"))

model.add(Dense(64, activation = "relu"))

model.add(Dense(5, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])



model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 510, 510, 10)      280       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 255, 255, 10)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 253, 253, 10)      910       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 126, 126, 10)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 124, 124, 10)      910       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 62, 10)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 60, 60, 10)        910       
__________

In [5]:
model.fit(x = x_train, y = y_trainOneHot, batch_size = 100, epochs = 20, validation_split = 0.1, verbose = 2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1080 samples, validate on 120 samples
Epoch 1/20
 - 429s - loss: 6.8065 - acc: 0.3167 - val_loss: 0.9639 - val_acc: 0.7250
Epoch 2/20
 - 463s - loss: 0.9721 - acc: 0.7222 - val_loss: 0.9839 - val_acc: 0.7333
Epoch 3/20
 - 462s - loss: 0.9090 - acc: 0.7352 - val_loss: 0.9209 - val_acc: 0.7333
Epoch 4/20
 - 463s - loss: 0.8747 - acc: 0.7185 - val_loss: 0.9189 - val_acc: 0.7333
Epoch 5/20
 - 449s - loss: 0.7995 - acc: 0.7352 - val_loss: 0.9003 - val_acc: 0.7333
Epoch 6/20
 - 454s - loss: 0.7351 - acc: 0.7463 - val_loss: 0.9815 - val_acc: 0.7333
Epoch 7/20
 - 458s - loss: 0.6877 - acc: 0.7509 - val_loss: 1.0495 - val_acc: 0.7167
Epoch 8/20
 - 458s - loss: 0.6215 - acc: 0.7741 - val_loss: 1.0411 - val_acc: 0.7000
Epoch 9/20
 - 457s - loss: 0.5800 - acc: 0.7944 - val_loss: 0.9624 - val_acc: 0.7083
Epoch 10/20
 - 469s - loss: 0.6227 - acc: 0.8019 - val_loss: 1.0698 - val_acc: 0.6833
Epoch 1

In [7]:
scores = model.evaluate(x_test, y_testOneHot)
scores[1]

300/300 [==============================] - 24s 81ms/step


0.64

In [8]:

prediction = model.predict_classes(x_test)
pd.crosstab(y_test,prediction,
            rownames=['label'],colnames=['predict'])

predict,0,1,2,3
label,,,,
0,184,15,13,1
1,23,1,0,0
2,42,3,7,0
3,5,0,3,0
4,3,0,0,0
